In [1]:
import time
import re
import pandas as pd
from datetime import datetime

# --- BIBLIOTECAS NECESSÁRIAS PARA SELENIUM ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

# ==============================================================================
# CONFIGURAÇÕES GLOBAIS
# ==============================================================================

BASE_URL = "https://www.dfimoveis.com.br/aluguel/df/todos/apartamento"

# Headers robustos para contornar o 403 (baseado na sua inspeção)
USER_AGENT_ROBUSTO = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'
NUM_CLEANER = re.compile(r'[^\d,\.]+')

# ==============================================================================
# FUNÇÕES DE UTILIDADE E REQUISIÇÃO
# ==============================================================================

def clean_numeric_value(text, is_float=False):
    """Remove caracteres não numéricos e converte para float/int."""
    if not text:
        return None
    cleaned = text.replace('R$', '').replace('.', '').replace('m²', '').strip()
    cleaned = cleaned.replace(',', '.') 
    
    # Remove tudo que não for dígito ou ponto decimal, mas só se houver dígitos
    final_cleaned = re.sub(r'[^\d\.]', '', cleaned)
    
    try:
        # Se for inteiro, usamos int(float()) para lidar com strings como '1.0'
        if is_float:
            return float(final_cleaned)
        return int(float(final_cleaned)) 
    except (ValueError, TypeError):
        return None

def fetch_html_content_selenium(url):
    """Usa Selenium para obter o HTML renderizado."""
    print(f"\t-> Buscando URL via Selenium: {url}")
    
    chrome_options = Options()
    # Modo Headless é crucial para rodar em servidores
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f"user-agent={USER_AGENT_ROBUSTO}") 

    driver = None
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        driver.get(url)
        time.sleep(5) # Espera para carregar o JS
        
        html_content = driver.page_source
        
        if "captcha" in driver.current_url.lower() or "blocked" in html_content.lower():
             raise RequestException("Cloudflare CAPTCHA detectado.")
             
        return html_content

    except Exception as e:
        print(f"\tERRO no acesso via Selenium: {e}")
        return None
    finally:
        if driver:
            driver.quit()

# ==============================================================================
# FUNÇÃO DE EXTRAÇÃO DE DETALHES (PÁGINA INDIVIDUAL)
# ==============================================================================

def extract_detailed_data(html_content, link):
    """Extrai todas as informações ricas da página de detalhes do imóvel."""
    soup = BeautifulSoup(html_content, 'html.parser')
    data = {'link': link}

    # --- 1. PREÇO, ÁREA e CUSTOS FIXOS ---
    try:
        # Preço do aluguel (H4 dentro de div com classe .exibirPrecoSalao)
        preco_element = soup.select_one('.exibirPrecoSalao .headline-medium')
        data['preco_aluguel_rs'] = clean_numeric_value(preco_element.text, is_float=True) if preco_element else None
        
        # Área Útil (Valor no H4 próximo ao 'Área Útil:')
        area_tag = soup.find('span', string=lambda t: t and 'Área Útil:' in t)
        area_element = area_tag.find_next_sibling('h4') if area_tag else None
        data['area_m2'] = clean_numeric_value(area_element.text, is_float=True) if area_element else None

        # Condomínio
        cond_element = soup.select_one('.condom span.label-large.bold')
        raw_cond = re.sub(r'[A-Za-z\r\n$\s<>/br]+', '', cond_element.text).replace(',', '.') if cond_element else '0'
        data['condominio_rs'] = clean_numeric_value(raw_cond, is_float=True)

        # IPTU - CORREÇÃO AQUI
        data['iptu_rs'] = 0  # Valor padrão
        
        # Procura pelo <li> que contém "IPTU R$:"
        iptu_li = soup.find('li', string=lambda t: t and 'IPTU R$:' in t if t else False)
        
        if iptu_li:
            # Procura pelo <span> com classe grey-color dentro do mesmo <li>
            iptu_span = iptu_li.find('span', class_='grey-color')
            if iptu_span:
                data['iptu_rs'] = clean_numeric_value(iptu_span.text.strip(), is_float=True)
        
        # Fallback: se não encontrou pelo método acima, tenta buscar por texto direto
        if data['iptu_rs'] == 0:
            # Busca todos os <li> e verifica se contém "IPTU"
            all_lis = soup.find_all('li')
            for li in all_lis:
                li_text = li.get_text()
                if 'IPTU R$:' in li_text:
                    # Extrai o valor numérico após "IPTU R$:"
                    iptu_match = re.search(r'IPTU R\$:\s*(\d+(?:,\d+)?)', li_text)
                    if iptu_match:
                        data['iptu_rs'] = clean_numeric_value(iptu_match.group(1), is_float=True)
                        break

        # Valor R$/m²
        valor_m2_tag = soup.find('span', string=lambda t: t and 'Valor R$/m²:' in t)
        valor_m2_element = valor_m2_tag.find_next_sibling('h4') if valor_m2_tag else None
        data['valor_aluguel_m2_rs'] = clean_numeric_value(valor_m2_element.text, is_float=True) if valor_m2_element else None

    except Exception as e:
        print(f"\t\t- ERRO na extração de PREÇOS/CUSTOS: {e}")
        data['preco_aluguel_rs'] = None
        data['area_m2'] = None
        data['condominio_rs'] = 0
        data['iptu_rs'] = 0
    
    # --- 2. CARACTERÍSTICAS E ENDEREÇO ---
    try:
        # Endereço (SQS 203, ASA SUL, BRASILIA) - Extraído do cabeçalho
        local_header = soup.select_one('.imv-map h4.body-medium')
        if local_header:
            endereco_completo = local_header.text.strip()
            # Esperamos: UF - CIDADE - BAIRRO - LOGRADOURO
            partes = [p.strip() for p in endereco_completo.split(' - ')]
            data['cidade'] = partes[1] if len(partes) > 1 else None
            data['bairro'] = partes[2] if len(partes) > 2 else None
            data['logradouro'] = partes[3] if len(partes) > 3 else None
        
        # Quartos/Suítes (o texto contém a contagem)
        room_element = soup.select_one('.room span.label-large.bold')
        suite_element = soup.select_one('.suite span.label-large.bold')
        
        data['quartos'] = clean_numeric_value(room_element.text) if room_element else None
        data['suites'] = clean_numeric_value(suite_element.text) if suite_element else 0

    except Exception as e:
        print(f"\t\t- ERRO na extração de CARACTERÍSTICAS: {e}")

    # --- 3. GEOLOCALIZAÇÃO ---
    try:
        lat_script = soup.find('script', string=lambda t: t and 'latitude =' in t)
        if lat_script:
            lat = re.search(r'latitude = ([-]?[\d.]+);', lat_script.string).group(1)
            lon = re.search(r'longitude = ([-]?[\d.]+);', lat_script.string).group(1)
            data['latitude'] = float(lat)
            data['longitude'] = float(lon)
        else:
            data['latitude'] = None
            data['longitude'] = None
    except Exception as e:
        print(f"\t\t- ERRO na extração de GEOLOCALIZAÇÃO: {e}")
        data['latitude'] = None
        data['longitude'] = None

    # --- 4. DESCRIÇÃO (Para Features de NLP) ---
    descricao_element = soup.select_one('.assined-imv.w-100')
    data['descricao_texto'] = descricao_element.text.strip() if descricao_element else ''

    return data


In [2]:
# ==============================================================================
# FLUXO PRINCIPAL: VALIDAÇÃO DE 1 IMÓVEL
# ==============================================================================

def run_validation_scrape():
    """Executa o scraping da página de listagem, pega o primeiro link e raspa os detalhes."""
    
    timestamp = datetime.now()
    
    # 1. ACESSO À PÁGINA DE LISTAGEM (Para obter links)
    html_listagem = fetch_html_content_selenium(BASE_URL)
    
    if not html_listagem:
        print("\nFalha crítica na requisição da página de listagem. Encerrando.")
        return
        
    soup_listagem = BeautifulSoup(html_listagem, 'html.parser')
    
    # Encontra o PRIMEIRO card de imóvel
    primeiro_card = soup_listagem.find('a', class_='imovel-card')
    
    if not primeiro_card:
        print("\nNenhum imóvel encontrado na primeira página.")
        return
    
    # Extrai o link do primeiro imóvel
    link_primeiro_imovel = primeiro_card.get('href', 'N/A')
    if not link_primeiro_imovel.startswith('http'):
        link_primeiro_imovel = "https://www.dfimoveis.com.br" + link_primeiro_imovel
        
    print(f"\n[Validação] Link do Primeiro Imóvel: {link_primeiro_imovel}")
    
    # 2. ACESSO À PÁGINA DE DETALHE (O passo mais importante)
    html_detalhe = fetch_html_content_selenium(link_primeiro_imovel)
    
    if not html_detalhe:
        print("\nFalha na requisição da página de detalhe. Encerrando.")
        return

    # 3. EXTRAÇÃO DOS DADOS RICOS
    dados_detalhados = extract_detailed_data(html_detalhe, link_primeiro_imovel)

    # 4. IMPRESSÃO PARA VALIDAÇÃO
    print("\n" + "="*50)
    print("           RESULTADOS DA EXTRAÇÃO DETALHADA")
    print("="*50)
    
    for key, value in dados_detalhados.items():
        if key == 'descricao_texto':
            print(f"{key}: {value[:100]}...") # Imprime só os primeiros 100 caracteres
        else:
            print(f"{key}: {value}")
    print("="*50 + "\n")
    
    # 5. RETORNA UM DATAFRAME DE 1 LINHA
    df = pd.DataFrame([dados_detalhados])
    print("\n--- DataFrame Gerado para Validação ---")
    print(df[['cidade', 'bairro', 'logradouro', 'preco_aluguel_rs', 'condominio_rs', 'iptu_rs', 'valor_aluguel_m2_rs', 'area_m2', 'quartos', 'suites', 'latitude']].T)


In [3]:
# Teste rápido
if __name__ == '__main__':
    run_validation_scrape()

	-> Buscando URL via Selenium: https://www.dfimoveis.com.br/aluguel/df/todos/apartamento

[Validação] Link do Primeiro Imóvel: https://www.dfimoveis.com.br/imovel/apartamento-2-quartos-aluguel-asa-sul-brasilia-df-sqs-203-1240085
	-> Buscando URL via Selenium: https://www.dfimoveis.com.br/imovel/apartamento-2-quartos-aluguel-asa-sul-brasilia-df-sqs-203-1240085

           RESULTADOS DA EXTRAÇÃO DETALHADA
link: https://www.dfimoveis.com.br/imovel/apartamento-2-quartos-aluguel-asa-sul-brasilia-df-sqs-203-1240085
preco_aluguel_rs: 8200.0
area_m2: 119.0
condominio_rs: 1513.0
iptu_rs: 238.0
valor_aluguel_m2_rs: 68.0
cidade: BRASILIA
bairro: ASA SUL
logradouro: SQS 203
quartos: 2
suites: 2
latitude: -15.808588
longitude: -47.8887643
descricao_texto: Este lindo apartamento une sofisticação e funcionalidade em uma das localizações mais desejadas de B...


--- DataFrame Gerado para Validação ---
                             0
cidade                BRASILIA
bairro                 ASA SUL
logradou